In [ ]:
"""
Working with the eBird API.

This notebook show you how to create a database and load observations
from the eBird API v2.0.

You will need to sign up for a key to access the eBird API.
https://ebird.org/api/keygen

You will also need to copy the example file .env.example to .env,
and add your API key.
"""

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from ebird.notebooks import models, loaders, settings

# The URL for accessing the database
db_url = settings.DB_URL
# The key needed to access the API.
api_key = settings.API_KEY

In [ ]:
# Create and initialize the database.
engine = create_engine(db_url)
models.Base.metadata.create_all(engine)

In [ ]:
# The eBird API returns only the species code, e.g. "horlar1", so
# we need to populate the species table with the complete taxonomy.
with Session(engine) as session:
    loader = loaders.SpeciesLoader(api_key, session)
    loader.load()

In [ ]:
# Load the observations from the eBird API.
with Session(engine) as session:
    loader = loaders.APILoader(api_key, session)
    
    for region in settings.API_REGIONS:
        for date in settings.API_PAST_DATES:
            loader.load(region, date)

In [ ]:
# If you are running Linux or MacOS you can schedule the loading of
# checklists from the eBird API using cron. Here we get cron to run
# the load script at 9am, every day. Note: your crontab will be updated
# every time you execute this cell.

from crontab import CronTab  # noqa : E402
from pathlib import Path  # noqa : E402

python = Path(settings.ROOT_DIR, ".venv", "bin", "python")
script = Path(settings.ROOT_DIR, "scripts", "load.py")

with CronTab(user=True) as cron:
    job = cron.new(command=f"{python} {script}")
    job.hour.on(9)